In [28]:
import pandas as pd
import numpy as np

data = pd.read_csv("/content/drive/MyDrive/data_train.csv").to_numpy()

mat_total = np.zeros((10000, 1000))
mat_train = np.zeros((10000, 1000))
mat_validation = np.zeros((10000, 1000))

np.random.seed(42)
mask = np.random.rand(len(data[:, 0])) < 0.9

for i, row in enumerate(data):
  a, b = row[0].split("_")
  m, n = int(a[1:]) - 1, int(b[1:]) - 1
  mat_total[m, n] = row[1]

  if mask[i]:
    mat_train[m, n] = row[1]
  else:
    mat_validation[m, n] = row[1]

In [ ]:
%load_ext tensorboard

In [33]:
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

class UserBias(keras.layers.Layer):
  def __init__(self):
    super().__init__()
    self.bias = self.add_weight(shape=((10000,)), initializer="zeros", trainable=True)

  def call(self, inputs):
    biases = tf.gather(self.bias, inputs[1][:, 0])
    return inputs[0] + tf.transpose([biases])

def train_loss(y_true, y_pred):
  return tf.reduce_mean(((y_true - y_pred) ** 2) * tf.clip_by_value(y_true, 0, 1))

def val_loss(y_true, y_pred):
  mask = tf.clip_by_value(y_true, 0, 1)
  return tf.sqrt(tf.reduce_sum(((y_true - y_pred) ** 2) * mask) / tf.reduce_sum(mask))

ratings = keras.Input((1000,))
user_id = keras.Input((1,), dtype=tf.int32)

x = ratings
x = layers.Dropout(0.5)(x)
x = layers.Dense(12, "elu")(x)
x = layers.Dense(12, "elu")(x)
x = layers.Dense(12, "elu")(x)
x = layers.Dense(12, "elu")(x)
x = layers.Dense(12, "elu")(x)
x = layers.Dense(1000, "linear")(x)
x = UserBias()([x, user_id])

model = keras.Model(inputs=[ratings, user_id], outputs=[x])
model.add_loss(train_loss(ratings, x))

lr = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=10.0,
    decay_steps=800,
    decay_rate=0.96)

model.compile(optimizer=keras.optimizers.Adadelta(learning_rate=lr), metrics=[val_loss])

ids = np.asarray([np.arange(10000)]).T

mask = np.random.random((10000, 1000)) < 0.5
val = mat_total * mask

#tb_callback = tf.keras.callbacks.TensorBoard("enc")
model.fit([mat_train, ids], mat_train, epochs=300, batch_size=64, validation_data=([mat_validation, ids], mat_validation), shuffle=True)#, callbacks=[tb_callback])

Epoch 1/300
157/157 [==============================] - 2s 11ms/step - loss: 0.2784 - val_loss: 0.0244 - val_val_loss: 1.4396
Epoch 2/300
157/157 [==============================] - 2s 10ms/step - loss: 0.1184 - val_loss: 0.0138 - val_val_loss: 1.0831
Epoch 3/300
157/157 [==============================] - 1s 10ms/step - loss: 0.1165 - val_loss: 0.0151 - val_val_loss: 1.1308
Epoch 4/300
157/157 [==============================] - 1s 10ms/step - loss: 0.1139 - val_loss: 0.0139 - val_val_loss: 1.0864
Epoch 5/300
157/157 [==============================] - 2s 10ms/step - loss: 0.1139 - val_loss: 0.0131 - val_val_loss: 1.0542
Epoch 6/300
157/157 [==============================] - 1s 7ms/step - loss: 0.1124 - val_loss: 0.0143 - val_val_loss: 1.1024
Epoch 7/300
157/157 [==============================] - 2s 10ms/step - loss: 0.1114 - val_loss: 0.0142 - val_val_loss: 1.0980
Epoch 8/300
157/157 [==============================] - 1s 7ms/step - loss: 0.1110 - val_loss: 0.0138 - val_val_loss: 1.0832
Ep

In [34]:
print(val_loss(mat_validation, model.predict([mat_validation, ids])))

tf.Tensor(1.0452554375217302, shape=(), dtype=float64)


In [ ]:
%tensorboard --logdir enc

In [35]:
data_sample = pd.read_csv("/content/drive/MyDrive/sampleSubmission.csv").to_numpy()
target_indices = []
for d in data_sample:
    a, b = d[0].split("_")
    target_indices.append((int(a[1:]) - 1, int(b[1:]) - 1))

pred = model.predict([mat_total, ids])

with open("submission.csv", "w") as f:
  f.write("Id,Prediction\n")
  for t in target_indices:
    f.write(f"r{t[0]+1}_c{t[1]+1},{pred[t[0], t[1]]}\n")

import zipfile
zipfile.ZipFile("submission.zip", "w").write("submission.csv", compress_type=zipfile.ZIP_DEFLATED)